In [ ]:
pip install fuzzywuzzy

In [1]:
import json
import pandas as pd
import geopandas as gpd
from fuzzywuzzy import fuzz

with open('Data/unsupervised_places_interactions_ch3.json', 'r') as f:
    data = json.load(f)

json_df = pd.read_json('Data/unsupervised_places_interactions_ch3.json')

#display(data)
display(json_df)



/Users/danielebelfiore/anaconda3/envs/ada/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


,place,interaction,description
0,Jerusalem,visited,Visited and explored various localities in and...
1,Bethlehem,mentioned,Plans to visit Bethlehem and reflect on the bi...
2,Bethany,mentioned,Intends to visit Bethany and read about the re...
3,Gethsemane,mentioned,Looks forward to visiting Gethsemane to contem...
4,Calvary,mentioned,Aims to visit Calvary and reflect on the story...
5,Mount Olivet,mentioned,Anticipates visiting Mount Olivet to consider ...
6,Zion,observed,"Observed the hill of Zion, a significant locat..."
7,Akra,observed,Noted the hill of Akra while exploring Jerusalem.
8,Bezetha,observed,"Bezetha, one of the hills on which Jerusalem i..."
9,Moriah,observed,"Observed Moriah, another hill forming part of ..."


In [2]:
# Replace this with the path to your .shp file
shapefile_path = 'Data/landmarks_names.shp'

# Read the shapefile
gdf = gpd.read_file(shapefile_path)

# Now you can work with the GeoDataFrame 'gdf' as you would with a Pandas DataFrame
gdf  # This will print the first few rows of the GeoDataFrame

#gdf.plot("area", legend=True)
print(type(gdf['geometry'].iloc[0]))


<class 'shapely.geometry.point.Point'>


In [3]:
# Function to perform fuzzy matching
def fuzzy_match(toponym1, toponym2):
    return fuzz.token_set_ratio(str(toponym1), str(toponym2))

# Dictionary to store the best match for each place in json_df
best_matches = {}

# Iterate through rows in json_df and find matches in gdf
for index1, row1 in json_df.iterrows():
    place_in_json = row1['place']

    # Initialize variables to track the best match for the current place_in_json
    best_match_score = 0
    best_match_place = None
    best_match_geometry = None

    # Iterate through rows in gdf
    for index2, row2 in gdf.iterrows():
        # Iterate through language columns in gdf
        for lang_column in ["English", "Arabic", "English_2", "Arabic_2", "German", "Hebrew"]:
            place_in_gdf = row2[lang_column]

            # Skip if the language column is None
            if pd.notnull(place_in_gdf):
                similarity_score = fuzzy_match(place_in_json, place_in_gdf)
                
                # Update the best match if the current match has a higher similarity score
                if similarity_score >= 80 and similarity_score > best_match_score:
                    best_match_score = similarity_score
                    best_match_place = place_in_gdf
                    best_match_geometry = row2['geometry']

    # Store the best match for the current place_in_json in the dictionary
    if best_match_place is not None:
        best_matches[place_in_json] = {
            'place_in_json': place_in_json,
            'place_in_gdf': best_match_place,
            'similarity_score': best_match_score,
            'geometry': best_match_geometry,
            'description': row1['description']
        }

# Create a dataframe from the dictionary of best matches
best_matches_df = pd.DataFrame(list(best_matches.values()))

# Convert the best_matches_df to a GeoDataFrame
best_matches_gdf = gpd.GeoDataFrame(best_matches_df, geometry='geometry')

# Display or further process the best_matches_gdf as needed
print(best_matches_gdf)


                   place_in_json                   place_in_gdf  \
0                     Gethsemane                     Gethsemane   
1                           Zion                      Zion Gate   
2                  Damascus Gate        Damascus Gate\t\t\t\t\t   
3             St. Stephen's Gate             St. Stephen's Gate   
4                    Golden Gate                    Golden Gate   
5                      Zion Gate                      Zion Gate   
6                     Jaffa Gate                     Jaffa Gate   
7                 Pool of Siloam                 Pool of Siloam   
8            Church of St. James             Church of St. Anne   
9                  Tomb of David                   David's Tomb   
10                  Via Dolorosa                   Via Dolorosa   
11            Church of St. Anne             Church of St. Anne   
12  Church of the Holy Sepulchre   Church of the Holy Sepulchre   
13        Church of Flagellation  Monastery of the Flagellatio

In [4]:
import pandas as pd
from shapely import wkt

# Replace 'FILEPATH' with the actual file path of the Excel file
df = pd.read_excel('Data/final.xlsx')
def convert_to_point(cell):
    if not pd.isnull(cell):
        return wkt.loads(str(cell))
    return None

# Apply the function to the geometry column
df['Geometry'] = df['Geometry'].apply(convert_to_point)

print(df)
print(type(df['Geometry'].iloc[0]))
final_gdf = gpd.GeoDataFrame(df, geometry='Geometry')

# Display the DataFrame
display(final_gdf)



      no  Location JSON Name Matched Name  Chapter  Matched interaction  \
0      1   Mediterranean Sea          NaN        1    False    observed   
1      2               Malta          NaN        1    False   mentioned   
2      3              Beirût          NaN        1    False     visited   
3      4   La Bellevue Hotel          NaN        1    False      stayed   
4      5             Lebanon          NaN        1    False    observed   
..   ...                 ...          ...      ...      ...         ...   
165  166           Jerusalem          NaN       20    False    observed   
166  167           Bethlehem          NaN       20    False   mentioned   
167  168  Gardens of Solomon          NaN       20    False   mentioned   
168  169        Mount Olivet          NaN       20    False    observed   
169  170               Syria          NaN       20    False     visited   

                                           description  \
0    Caught the first glimpse of land whi

,no,Location JSON Name,Matched Name,Chapter,Matched,interaction,description,Wiki URL,Wiki URL + Jerusalem,Wiki Name,Image,Geometry,coordinate_added,Unnamed: 13,img
0,1,Mediterranean Sea,NaN,1,False,observed,Caught the first glimpse of land while crossin...,https://en.wikipedia.org/wiki/Mediterranean_Sea,https://en.wikipedia.org/wiki/Mediterranean_Sea,Mediterranean Sea,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN
1,2,Malta,NaN,1,False,mentioned,Left their barque at Malta and took passage on...,https://en.wikipedia.org/wiki/Malta,https://en.wikipedia.org/wiki/Sovereign_Milita...,Malta,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN
2,3,Beirût,NaN,1,False,visited,"Visited Beirût, observing its white terraced h...",https://en.wikipedia.org/wiki/Beirut,https://en.wikipedia.org/wiki/From_Beirut_to_J...,Beirut,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN
3,4,La Bellevue Hotel,NaN,1,False,stayed,Secured lodgings at 'La Bellevue' hotel in Bei...,https://en.wikipedia.org/wiki/The_Bellevue-Str...,NaN,The Bellevue-Stratford Hotel,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN
4,5,Lebanon,NaN,1,False,observed,Admired the snowy peaks of Lebanon from Beirût.,https://en.wikipedia.org/wiki/Lebanon,https://en.wikipedia.org/wiki/Anglican_Diocese...,Lebanon,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,166,Jerusalem,NaN,20,False,observed,Observed the mild winters and enjoyed summer b...,https://en.wikipedia.org/wiki/Jerusalem,https://en.wikipedia.org/wiki/Jerusalem,Jerusalem,https://upload.wikimedia.org/wikipedia/commons...,POINT (35.22556 31.77889),1.0,NaN,1.0
166,167,Bethlehem,NaN,20,False,mentioned,Mentioned Bethlehem in the context of the Gard...,https://en.wikipedia.org/wiki/Bethlehem,https://en.wikipedia.org/wiki/Bethlehem,Bethlehem,https://upload.wikimedia.org/wikipedia/commons...,POINT (35.20639 31.70444),1.0,NaN,1.0
167,168,Gardens of Solomon,NaN,20,False,mentioned,Referred to the Gardens of Solomon near Bethle...,https://en.wikipedia.org/wiki/Solomons_cockatoo,https://en.wikipedia.org/wiki/Seal_of_Solomon,Solomons cockatoo,https://upload.wikimedia.org/wikipedia/commons...,None,NaN,NaN,NaN
168,169,Mount Olivet,NaN,20,False,observed,"Observed Mount Olivet, noting its plantation o...",https://en.wikipedia.org/wiki/Mount_of_Olives,https://en.wikipedia.org/wiki/Mount_of_Olives,Mount of Olives,https://upload.wikimedia.org/wikipedia/commons...,POINT (35.24389 31.77833),1.0,NaN,1.0


In [5]:
print(final_gdf['Chapter'].unique())


[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 18 19 20]


In [6]:
import plotly.express as px
import plotly.graph_objs as go
import geopandas as gpd
import math
import seaborn as sns

# Assuming 'best_matches_gdf' is your GeoDataFrame
final_gdf['longitude'] = final_gdf.geometry.x
final_gdf['latitude'] = final_gdf.geometry.y


# Create initial scatter plot
fig = px.scatter_geo(final_gdf, lat='latitude', lon='longitude', hover_name='description', text='Location JSON Name')
fig.update_traces(textposition='bottom center')

mid_lons = []
mid_lats = []


def calculate_bearing(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Difference in coordinates
    d_lon = lon2 - lon1

    # Calculate bearing
    x = math.sin(d_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))

    # Obtain bearing in degrees
    bearing = math.degrees(math.atan2(x, y))

    # Normalize the bearing
    bearing = (bearing + 360) % 360

    return bearing

chapter_colors = sns.color_palette("hls", final_gdf['Chapter'].unique()[-1]).as_hex()


# Add lines and annotations
for i in range(len(final_gdf) - 1):
    
    lon1, lat1 = final_gdf['longitude'].iloc[i], final_gdf['latitude'].iloc[i]
    lon2, lat2 = final_gdf['longitude'].iloc[i+1], final_gdf['latitude'].iloc[i+1]

    angle = 0
    mid_lon = (lon1 + lon2) / 2
    mid_lat = (lat1 + lat2) / 2
    bearing = calculate_bearing(lat1, lon1, lat2, lon2)


    
    # Adding lines
    fig.add_trace(go.Scattergeo(
        lon=[lon1, lon2],
        lat=[lat1, lat2],
        mode='lines+text',
        name=i+1,
        line=dict(width=2, color=chapter_colors[final_gdf['Chapter'].iloc[i]-1]),
        hovertext=final_gdf['description'].iloc[i],
        opacity=1,
        showlegend=False
        
        )
    )

    fig.add_trace(go.Scattergeo(
        lon=[mid_lon],
        lat=[mid_lat],
        mode='text+markers',
        name=i+1,
        hovertext=final_gdf['description'].iloc[i],
        opacity=1,
        showlegend=False,


        text="",  # Assuming 'label' is a column with labels for lines
        textposition='middle center',
        textfont=dict(
        size=14,
        color='red',  # Change to your preferred color
        family='Arial'  # Specify font family and style
    ),
        
        marker=dict(
        color=chapter_colors[final_gdf['Chapter'].iloc[i]-1],  # Black background
        opacity=1,
        size=14,
        symbol='arrow-wide',
        angle=bearing,
    ),
        )
    )

    #mid_lons.append(mid_lon)
    #mid_lats.append(mid_lat)

    #px.scatter(mid_lats, mid_lons, text='pino')
    








fig.update_geos(fitbounds="locations")

# Show the figure
fig.show()


In [7]:
import plotly.express as px
import plotly.graph_objs as go
import geopandas as gpd
import math
import seaborn as sns

# Assuming 'best_matches_gdf' is your GeoDataFrame
final_gdf['longitude'] = final_gdf.geometry.x
final_gdf['latitude'] = final_gdf.geometry.y

def calculate_bearing(lat1, lon1, lat2, lon2):
        # Convert latitude and longitude from degrees to radians
        lat1 = math.radians(lat1)
        lon1 = math.radians(lon1)
        lat2 = math.radians(lat2)
        lon2 = math.radians(lon2)

        # Difference in coordinates
        d_lon = lon2 - lon1

        # Calculate bearing
        x = math.sin(d_lon) * math.cos(lat2)
        y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))

        # Obtain bearing in degrees
        bearing = math.degrees(math.atan2(x, y))

        # Normalize the bearing
        bearing = (bearing + 360) % 360

        return bearing


for chapter in final_gdf['Chapter'].unique():

    framed_gdf = final_gdf[final_gdf['Chapter'] == chapter]
    print(framed_gdf['coordinate_added'][0] is nan )
    display(framed_gdf)

    # Create initial scatter plot
    fig = px.scatter_geo(framed_gdf, lat='latitude', lon='longitude', hover_name='description', text='Location JSON Name')
    fig.update_traces(textposition='bottom center')

    mid_lons = []
    mid_lats = []


    

    chapter_colors = sns.color_palette("hls", final_gdf['Chapter'].unique()[-1]).as_hex()


    # Add lines and annotations
    for i in range(len(framed_gdf) - 1):
        
        lon1, lat1 = final_gdf['longitude'].iloc[i], final_gdf['latitude'].iloc[i]
        lon2, lat2 = final_gdf['longitude'].iloc[i+1], final_gdf['latitude'].iloc[i+1]

        angle = 0
        mid_lon = (lon1 + lon2) / 2
        mid_lat = (lat1 + lat2) / 2
        bearing = calculate_bearing(lat1, lon1, lat2, lon2)


        
        # Adding lines
        fig.add_trace(go.Scattergeo(
            lon=[lon1, lon2],
            lat=[lat1, lat2],
            mode='lines+text',
            name=i+1,
            line=dict(width=2, color=chapter_colors[framed_gdf['Chapter'].iloc[i]-1]),
            hovertext=framed_gdf['description'].iloc[i],
            opacity=1,
            showlegend=False
            
            )
        )

        fig.add_trace(go.Scattergeo(
            lon=[mid_lon],
            lat=[mid_lat],
            mode='text+markers',
            name=i+1,
            hovertext=framed_gdf['description'].iloc[i],
            opacity=1,
            showlegend=False,


            text="",  # Assuming 'label' is a column with labels for lines
            textposition='middle center',
            textfont=dict(
            size=14,
            color='red',  # Change to your preferred color
            family='Arial'  # Specify font family and style
        ),
            
            marker=dict(
            color=chapter_colors[framed_gdf['Chapter'].iloc[i]-1],  # Coloured background
            opacity=1,
            size=14,
            symbol='arrow-wide',
            angle=bearing,
        ),
            )
        )

        #mid_lons.append(mid_lon)
        #mid_lats.append(mid_lat)

        #px.scatter(mid_lats, mid_lons, text='pino')
        








    fig.update_geos(fitbounds="locations")

    # Show the figure
    fig.show()


NameError: name 'nan' is not defined

In [8]:
import plotly.express as px
import plotly.graph_objs as go
import math
import seaborn as sns
import geopandas as gpd

# Your GeoDataFrame
# final_gdf = ...

def calculate_bearing(lat1, lon1, lat2, lon2):
        # Convert latitude and longitude from degrees to radians
        lat1 = math.radians(lat1)
        lon1 = math.radians(lon1)
        lat2 = math.radians(lat2)
        lon2 = math.radians(lon2)

        # Difference in coordinates
        d_lon = lon2 - lon1

        # Calculate bearing
        x = math.sin(d_lon) * math.cos(lat2)
        y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))

        # Obtain bearing in degrees
        bearing = math.degrees(math.atan2(x, y))

        # Normalize the bearing
        bearing = (bearing + 360) % 360

        return bearing

# Generate a color for each unique chapter
unique_chapters = final_gdf['Chapter'].unique()
chapter_colors = sns.color_palette("hls", len(unique_chapters)).as_hex()

# Create a mapping from chapter to color
chapter_to_color = {chapter: color for chapter, color in zip(unique_chapters, chapter_colors)}

# Calculate bearings and plot each chapter's path
for chapter in final_gdf['Chapter'].unique():
    framed_gdf = final_gdf[final_gdf['Chapter'] == chapter].copy().reset_index()
    
    # Create initial scatter plot for each chapter
    fig = px.scatter_geo(framed_gdf, lat='latitude', lon='longitude', hover_name='description', text='Location JSON Name')
    fig.update_traces(textposition='bottom center')

    # Loop through the rows of the chapter-specific GeoDataFrame
    for i in range(len(framed_gdf) - 1):
        lon1, lat1 = framed_gdf['longitude'].iloc[i], framed_gdf['latitude'].iloc[i]
        lon2, lat2 = framed_gdf['longitude'].iloc[i+1], framed_gdf['latitude'].iloc[i+1]
        bearing = calculate_bearing(lat1, lon1, lat2, lon2)

        # Adding lines
        fig.add_trace(go.Scattergeo(
            lon=[lon1, lon2],
            lat=[lat1, lat2],
            mode='lines+text',
            text='',
            line=dict(width=2, color=chapter_to_color[chapter]),
            showlegend=False
        ))
        # Optionally, add a midpoint marker for the arrow
        mid_lon = (lon1 + lon2) / 2
        mid_lat = (lat1 + lat2) / 2
        fig.add_trace(go.Scattergeo(
            lon=[mid_lon],
            lat=[mid_lat],
            mode='markers',
            marker=dict(size=14, color=chapter_to_color[chapter], symbol='arrow-wide', angle=bearing),
            showlegend=False
        ))

    fig.update_geos(fitbounds="locations")
    fig.update_layout(title=f'Journey for Chapter {chapter}')
    
    # Show the figure for each chapter
    fig.show()


In [20]:
import plotly.express as px
import plotly.graph_objs as go
import math
import seaborn as sns
import geopandas as gpd

# Your GeoDataFrame
# final_gdf = ...

def calculate_bearing(lat1, lon1, lat2, lon2):
        # Convert latitude and longitude from degrees to radians
        lat1 = math.radians(lat1)
        lon1 = math.radians(lon1)
        lat2 = math.radians(lat2)
        lon2 = math.radians(lon2)

        # Difference in coordinates
        d_lon = lon2 - lon1

        # Calculate bearing
        x = math.sin(d_lon) * math.cos(lat2)
        y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))

        # Obtain bearing in degrees
        bearing = math.degrees(math.atan2(x, y))

        # Normalize the bearing
        bearing = (bearing + 360) % 360

        return bearing

# Generate a color for each unique chapter
unique_chapters = final_gdf['Chapter'].unique()
chapter_colors = sns.color_palette("hls", len(unique_chapters)).as_hex()

# Create a mapping from chapter to color
chapter_to_color = {chapter: color for chapter, color in zip(unique_chapters, chapter_colors)}


mapbox_access_token = 'pk.eyJ1IjoiZGFuaWVsZS1iIiwiYSI6ImNscWNtbGxsYjAzbm0yam1zOGtyNDFneXUifQ.3d3-jVD6Ru81f8-D9EnGeA'

# Set the Mapbox style
mapbox_style = "mapbox://styles/mapbox/streets-v11"  # Example style, you can choose others


# Calculate bearings and plot each chapter's path
for chapter in final_gdf['Chapter'].unique():
    framed_gdf = final_gdf[final_gdf['Chapter'] == chapter].copy().reset_index()
    
    # Create initial scatter plot for each chapter
    
    #fig = px.scatter_geo(framed_gdf, lat='latitude', lon='longitude', hover_name='description', text='Location JSON Name')
    
    fig = go.Figure(go.Scattermapbox(
        lat=framed_gdf['latitude'],
        lon=framed_gdf['longitude'],
        mode='markers+text',
        marker=go.scattermapbox.Marker(size=15, color=chapter_to_color[chapter]),
        text=framed_gdf['Location JSON Name'],  # Text is the location name displayed on the markers
        hovertext=framed_gdf['description'],  # Hover text is the description
        showlegend=False,
        hoverinfo='text'  # Show hover text
    ))

    # Add lines between points
   
    
    
    fig.update_traces(textposition='bottom center')

    # Loop through the rows of the chapter-specific GeoDataFrame
    for i in range(len(framed_gdf) - 1):
        lon1, lat1 = framed_gdf['longitude'].iloc[i], framed_gdf['latitude'].iloc[i]
        lon2, lat2 = framed_gdf['longitude'].iloc[i+1], framed_gdf['latitude'].iloc[i+1]
        bearing = calculate_bearing(lat1, lon1, lat2, lon2)

        fig.add_trace(go.Scattermapbox(
            lat=[framed_gdf['latitude'].iloc[i], framed_gdf['latitude'].iloc[i+1]],
            lon=[framed_gdf['longitude'].iloc[i], framed_gdf['longitude'].iloc[i+1]],
            mode='lines',
            showlegend=False,
            line=go.scattermapbox.Line(width=2, color=chapter_to_color[chapter]),
            hoverinfo='skip'
        ))


        # Optionally, add a midpoint marker for the arrow
        mid_lon = (lon1 + lon2) / 2
        mid_lat = (lat1 + lat2) / 2

        fig.add_trace(go.Scattermapbox(
            lat=[mid_lat],
            lon=[mid_lon],
            mode='markers+text',
            showlegend=False,
            marker=go.scattermapbox.Marker(
                size=12,
                color='white',
            ),
            textposition='middle center',
            text=i+1,
            textfont=dict(
                size=10,
                color='black',  # Change to your preferred color
                family='Arial'  # Specify font family and style
            ),
            hoverinfo='none'
        ))


    fig.update_geos(fitbounds="locations")
    fig.update_layout(
            mapbox=dict(
                accesstoken=mapbox_access_token,
                style=mapbox_style,
                center=go.layout.mapbox.Center(lat=31.7683, lon=35.2137),  # Center on Jerusalem
                zoom=10  # Initial zoom level
            ),
            title=f'Journey for Chapter {chapter}'
        )    
    # Show the figure for each chapter
    fig.show()
    fig.write_html("Data/Figures/Plotly_" + str(chapter) + ".html")



In [ ]:
import plotly.graph_objs as go

# Assuming you have already set the mapbox_access_token variable

# Debugging function to print out the data and plot individual points
def debug_chapter(framed_gdf, chapter):
    print(f"Chapter {chapter} data:\n", framed_gdf[['latitude', 'longitude']])
    
    fig = go.Figure(go.Scattermapbox(
        lat=framed_gdf['latitude'],
        lon=framed_gdf['longitude'],
        mode='markers',
        marker=go.scattermapbox.Marker(size=9),
        text=framed_gdf['Location JSON Name'],
        hoverinfo='text'
    ))

    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style='mapbox://styles/mapbox/streets-v11',
            center=dict(lat=31.7683, lon=35.2137),  # Adjust as needed
            zoom=10  # Adjust as needed
        ),
        title=f'Debugging Journey for Chapter {chapter}'
    )

    fig.show()

# Apply the debugging function to chapters 1 and 2
for chapter in [1, 2]:
    framed_gdf = final_gdf[final_gdf['Chapter'] == chapter].copy().reset_index()
    debug_chapter(framed_gdf, chapter)


In [ ]:
import plotly.graph_objs as go

# Assuming you have already set the mapbox_access_token variable

# Debugging function to plot individual points with hover text
def debug_chapter(framed_gdf, chapter):
    print(f"Chapter {chapter} data:\n", framed_gdf[['latitude', 'longitude', 'description']])
    
    fig = go.Figure(go.Scattermapbox(
        lat=framed_gdf['latitude'],
        lon=framed_gdf['longitude'],
        mode='markers+text',  # Include text for the hover information and static text
        marker=go.scattermapbox.Marker(size=9),
        text=framed_gdf['Location JSON Name'],  # Static text always visible on the map
        hovertext=[  # Hover text shows description and coordinates
            f"{desc}<br>Lat: {lat}<br>Lon: {lon}"
            for desc, lat, lon in zip(framed_gdf['description'], framed_gdf['latitude'], framed_gdf['longitude'])
        ],
        hoverinfo='text'  # Show text on hover
    ))

    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style='mapbox://styles/mapbox/streets-v11',
            center=dict(lat=31.7683, lon=35.2137),  # Adjust as needed
            zoom=10  # Adjust as needed
        ),
        title=f'Debugging Journey for Chapter {chapter}',
        showlegend=False
    )

    fig.show()

# Apply the debugging function to chapters 1 and 2
for chapter in [1, 2]:
    framed_gdf


In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import math
import seaborn as sns
import geopandas as gpd

# Your GeoDataFrame
# final_gdf = ...

def calculate_bearing(lat1, lon1, lat2, lon2):
        # Convert latitude and longitude from degrees to radians
        lat1 = math.radians(lat1)
        lon1 = math.radians(lon1)
        lat2 = math.radians(lat2)
        lon2 = math.radians(lon2)

        # Difference in coordinates
        d_lon = lon2 - lon1

        # Calculate bearing
        x = math.sin(d_lon) * math.cos(lat2)
        y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))

        # Obtain bearing in degrees
        bearing = math.degrees(math.atan2(x, y))

        # Normalize the bearing
        bearing = (bearing + 360) % 360

        return bearing

# Generate a color for each unique chapter
unique_chapters = final_gdf['Chapter'].unique()
chapter_colors = sns.color_palette("hls", len(unique_chapters)).as_hex()

# Create a mapping from chapter to color
chapter_to_color = {chapter: color for chapter, color in zip(unique_chapters, chapter_colors)}


# Calculate global bounds for all chapters
all_bounds = final_gdf.total_bounds  # Returns (minx, miny, maxx, maxy)
global_lon_range = [all_bounds[0], all_bounds[2]]
global_lat_range = [all_bounds[1], all_bounds[3]]

# Define margins as a fraction of the total range
global_lon_margin = (global_lon_range[1] - global_lon_range[0]) * 0.1  # 10% of the longitude range
global_lat_margin = (global_lat_range[1] - global_lat_range[0]) * 0.1  # 10% of the latitude range

# Expand the global bounds by the margin
global_lon_range_with_margin = [(global_lon_range[0] - global_lon_margin), global_lon_range[1] + global_lon_margin]
global_lat_range_with_margin = [global_lat_range[0] - global_lat_margin, global_lat_range[1] + global_lat_margin]

# Set the layout of the figure to achieve a 16:9 aspect ratio
fig_width = 900
fig_height = int(fig_width / (16 / 9))  # Keep the 16:9 aspect ratio

# Calculate center of all points to center the map
center_lon = (global_lon_range_with_margin[0] + global_lon_range_with_margin[1]) / 2
center_lat = (global_lat_range_with_margin[0] + global_lat_range_with_margin[1]) / 2


# Calculate bearings and plot each chapter's path
for chapter in final_gdf['Chapter'].unique():
    framed_gdf = final_gdf[final_gdf['Chapter'] == chapter].copy().reset_index()
    
    # Create initial scatter plot for each chapter
    fig = px.scatter_geo(framed_gdf, lat='latitude', lon='longitude', hover_name='description', text='Location JSON Name')
    fig.update_traces(textposition='bottom center')
    fig_height = 450  # Example height, adjust as needed
    fig_width = int(fig_height * (21/9))  # Calculate width based on a 16:9 aspect ratio
    fig.update_layout(
        width=fig_width,
        height=fig_height,
        title=f'Journey for Chapter {chapter}'
    )
    # Loop through the rows of the chapter-specific GeoDataFrame
    for i in range(len(framed_gdf) - 1):
        lon1, lat1 = framed_gdf['longitude'].iloc[i], framed_gdf['latitude'].iloc[i]
        lon2, lat2 = framed_gdf['longitude'].iloc[i+1], framed_gdf['latitude'].iloc[i+1]
        bearing = calculate_bearing(lat1, lon1, lat2, lon2)

        # Adding lines
        fig.add_trace(go.Scattergeo(
            lon=[lon1, lon2],
            lat=[lat1, lat2],
            mode='lines+text',
            text='',
            line=dict(width=2, color=chapter_to_color[chapter]),
            showlegend=False
        ))
        # Optionally, add a midpoint marker for the arrow
        mid_lon = (lon1 + lon2) / 2
        mid_lat = (lat1 + lat2) / 2
        fig.add_trace(go.Scattergeo(
            lon=[mid_lon],
            lat=[mid_lat],
            mode='markers',
            marker=dict(size=14, color=chapter_to_color[chapter], symbol='arrow-wide', angle=bearing),
            showlegend=False
        ))

    


    fig.update_layout(
        width=fig_width,
        height=fig_height,
        title=f'Journey for Chapter {chapter}',
        margin={"r":0,"t":0,"l":0,"b":0}
    )

    fig.update_geos(
        fitbounds="locations",
        visible=True  # Set the geographic component to false to not force fitting to data extent
    )
    
    # Show the figure for each chapter
    fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objs as go

# Replace 'your_file.xlsx' with the path to your Excel file
file_path = 'Data/consolidated_chapters.xlsx'

# Read the Excel file
df = pd.read_excel(file_path)

# Assuming 'chapter' is a categorical column you want to filter by
chapters = df['Chapter'].unique()

# Function to create table data for a selected chapter
def create_table_data(chapter):
    filtered_df = df[df['Chapter'] == chapter]
    return [go.Table(
        header=dict(values=['Place', 'Interaction', 'Description'],
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[filtered_df.place, filtered_df.interaction, filtered_df.description],
                   fill_color='lavender',
                   align='left')
    )]

# Initial table with the first chapter's data
fig = go.Figure(data=create_table_data(chapters[0]))

# Update the layout to add a dropdown menu
fig.update_layout(
    title=f'Interactive Data for Chapter: {chapters[0]}',
    updatemenus=[
        {
            'buttons': [
                {
                    'label': f"Chapter {ch}",
                    'method': 'restyle',
                    'args': [{'cells': {'values': [df[df['Chapter'] == ch]['place'], 
                                                   df[df['Chapter'] == ch]['interaction'], 
                                                   df[df['Chapter'] == ch]['description']]},
                              'title': f'Interactive Data for Chapter: {ch}'}]
                } for ch in chapters
            ],
            'direction': 'down',
            'showactive': True,
        }
    ]
)

fig.show()

In [ ]:
import plotly.graph_objs as go
import pandas as pd

# Mockup dataframe similar to what you might have
df = pd.DataFrame({
    'Chapter': [1, 1, 2, 2],
    'Place': ['Jerusalem', 'Bethlehem', 'Bethany', 'Gethsemane'],
    'Interaction': ['visited', 'mentioned', 'mentioned', 'mentioned'],
    'Description': [
        'Visited and explored various localities in and around Jerusalem.',
        'Plans to visit Bethlehem and reflect on the birth of Christ.',
        'Intends to visit Bethany and read about the resurrection of Lazarus.',
        'Looks forward to visiting Gethsemane to contemplate Christ\'s agony.'
    ]
})

# Get a list of unique chapters
chapters = df['Chapter'].unique()

# Create the initial figure with the first chapter's data
initial_chapter = chapters[0]
initial_data = df[df['Chapter'] == initial_chapter]
fig = go.Figure(data=[go.Table(
    header=dict(values=['Place', 'Interaction', 'Description']),
    cells=dict(values=[initial_data['Place'], initial_data['Interaction'], initial_data['Description']])
)])

# Create the dropdown menu options
dropdown_buttons = [
    {
        'label': f"Chapter {ch}",
        'method': 'update',
        'args': [
            {'type': 'restyle',
             'header': {'values': ['Place', 'Interaction', 'Description']},
             'cells': {'values': [
                 df[df['Chapter'] == ch]['Place'],
                 df[df['Chapter'] == ch]['Interaction'],
                 df[df['Chapter'] == ch]['Description']
             ]}
            },
            {'title': f'Interactive Data for Chapter: {ch}'}
        ]
    } for ch in chapters
]

# Add dropdown to the figure layout
fig.update_layout(
    updatemenus=[
        {
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
        }
    ],
    title_text=f'Interactive Data for Chapter: {initial_chapter}'
)

# Show the figure
fig.show()


In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import math
import seaborn as sns
import geopandas as gpd

# Assuming 'final_gdf' is your GeoDataFrame
final_gdf['longitude'] = final_gdf.geometry.x
final_gdf['latitude'] = final_gdf.geometry.y

# Create initial scatter plot
fig = px.scatter_geo(final_gdf, lat='latitude', lon='longitude', hover_name='description', text='Location JSON Name')
fig.update_traces(textposition='bottom center')

def calculate_bearing(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Difference in coordinates
    d_lon = lon2 - lon1

    # Calculate bearing
    x = math.sin(d_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))

    # Obtain bearing in degrees
    bearing = math.degrees(math.atan2(x, y))

    # Normalize the bearing
    bearing = (bearing + 360) % 360

    return bearing

chapter_colors = sns.color_palette("hls", final_gdf['Chapter'].unique()[-1]).as_hex()

# Add lines and annotations
for i in range(len(final_gdf) - 1):
    lon1, lat1 = final_gdf['longitude'].iloc[i], final_gdf['latitude'].iloc[i]
    lon2, lat2 = final_gdf['longitude'].iloc[i+1], final_gdf['latitude'].iloc[i+1]
    bearing = calculate_bearing(lat1, lon1, lat2, lon2)
    chapter = final_gdf['Chapter'].iloc[i]
    chapter_color = chapter_colors[chapter - 1]

    # Adding lines
    fig.add_trace(go.Scattergeo(
        lon=[lon1, lon2],
        lat=[lat1, lat2],
        mode='lines',
        line=dict(width=2, color=chapter_color),
        name=f"Chapter {chapter}",
        legendgroup=f"chapter{chapter}",
        showlegend=True if i == 0 else False,  # Only show legend once per chapter
        customdata=[f"Chapter {chapter}"],  # Store chapter info for interactivity
        hoverinfo='none'
    ))

    # Adding arrows
    fig.add_trace(go.Scattergeo(
        lon=[(lon1 + lon2) / 2],
        lat=[(lat1 + lat2) / 2],
        mode='markers',
        marker=dict(
            size=8,
            color=chapter_color,
            symbol='arrow-bar-up',
            angle=bearing,
            opacity=0.5
        ),
        hoverinfo='none',
        showlegend=False,
        legendgroup=f"chapter{chapter}"
    ))

# Update layout to show the legend
fig.update_layout(
    showlegend=True,
    legend_title_text='Chapters',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

# Function to update the visibility based on legend click
def update_chapter_visibility(trace, points, state):
    # Get chapter from customdata of clicked trace
    clicked_chapter = trace.customdata[0]

    # Update the opacity of all traces based on the clicked chapter
    for t in fig.data:
        if t.customdata is not None:
            t.opacity = 0.1 if t.customdata[0] != clicked_chapter else 1

# Bind the function to click event on each trace
for trace in fig.data:
    trace.on_click(update_chapter_visibility)

# Update the figure to fit the geo data and display
fig.update_geos(fitbounds="locations")
fig.show()


In [ ]:
import plotly.graph_objs as go
from ipywidgets import widgets

# Initialize FigureWidget with scatter points
fig = go.FigureWidget(
    data=[go.Scattergeo(
        lon=final_gdf['longitude'], 
        lat=final_gdf['latitude'], 
        hovertext=final_gdf['description'], 
        text=final_gdf['Location JSON Name'],
        mode='markers+text',
        marker=dict(size=10, color='blue'),
        textposition='bottom center'
    )]
)

# Update layout
fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
fig.update_geos(fitbounds="locations")

# Function to add lines to the FigureWidget
def add_lines(fig, final_gdf, chapter_colors):
    for i in range(len(final_gdf) - 1):
        lon1, lat1 = final_gdf['longitude'].iloc[i], final_gdf['latitude'].iloc[i]
        lon2, lat2 = final_gdf['longitude'].iloc[i+1], final_gdf['latitude'].iloc[i+1]
        chapter = final_gdf['Chapter'].iloc[i]
        fig.add_trace(
            go.Scattergeo(
                lon=[lon1, lon2],
                lat=[lat1, lat2],
                mode='lines',
                line=dict(width=2, color=chapter_colors[chapter-1]),
                hoverinfo='skip',
                opacity=0.1,  # Start with a low opacity
                customdata=[chapter],  # Store the chapter number in customdata
                showlegend=False
            )
        )

# Add lines to the figure
add_lines(fig, final_gdf, chapter_colors)

# Define event handler for clicks
def update_paths(trace, points, selector):
    # Get the chapter from the clicked point's customdata
    clicked_chapter = trace.customdata[points.point_inds[0]]
    
    # Update the opacity of the lines based on the clicked chapter
    for t in fig.data:
        if 'line' in t.mode:
            t.opacity = 0.1 if t.customdata != clicked_chapter else 1

# Assign the event handler to the scatter points
scatter = fig.data[0]
scatter.on_click(update_paths)

# Show the figure
fig


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Create a GeoDataFrame for Jerusalem
jerusalem_point = gpd.GeoDataFrame({'place_in_json': ['Jerusalem']},
                                   geometry=[Point(35.2354, 31.7766)])  # Coordinates for Jerusalem

# Get the bounding box of the best_matches_gdf
xmin, ymin, xmax, ymax = best_matches_gdf.total_bounds

# Set up the map extent
#buffer_distance = 0.001  # degrees of longitude and latitude to include around Jerusalem
#xmin, ymin, xmax, ymax = jerusalem_point.total_bounds
#xmin -= buffer_distance
#ymin -= buffer_distance
#xmax += buffer_distance
#ymax += buffer_distance

# Plot the best_matches_gdf on a map centered around the bounding box
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)

# Plot the map background (optional)
#gpd.datasets.get_path('naturalearth_lowres') provides a world map shapefile
# You may need to download the world shapefile or replace it with another local shapefile
#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world.plot(ax=ax, color='lightgray', edgecolor='black')

# Plot the best_matches_gdf points with reduced marker size
best_matches_gdf.plot(ax=ax, color='red', marker='o', markersize=20, alpha=0.7)

# Plot the Jerusalem point with a larger marker
jerusalem_point.plot(ax=ax, color='blue', marker='*', markersize=200, label='Jerusalem')

# Add labels with adjusted positions
for x, y, label in zip(best_matches_gdf.geometry.x, best_matches_gdf.geometry.y, best_matches_gdf['place_in_json']):
    ax.text(x, y, label, fontsize=8, ha='right', va='bottom', color='black')

# Add label for Jerusalem
ax.text(jerusalem_point.geometry.x[0], jerusalem_point.geometry.y[0], 'Jerusalem', fontsize=12, ha='right', va='bottom', color='black')

# Show the plot
plt.legend()
plt.show()


In [ ]:
conda install nbformat

In [ ]:
display(best_matches_gdf)

In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import math


# Import other necessary libraries like contextily or folium for the map



best_matches_gdf['longitude'] = best_matches_gdf.geometry.x
best_matches_gdf['latitude'] = best_matches_gdf.geometry.y

fig = go.Figure()


# Assuming 'df' is your GeoDataFrame with coordinates and place names
fig = px.scatter_geo(best_matches_gdf, lat='latitude', lon='longitude', hover_name='description', text='place_in_json')
fig.update_traces(textposition='bottom center')



# Add lines and triangle annotations
for i in range(len(best_matches_gdf) - 1):
    lon1, lat1 = best_matches_gdf['longitude'].iloc[i], best_matches_gdf['latitude'].iloc[i]
    lon2, lat2 = best_matches_gdf['longitude'].iloc[i+1], best_matches_gdf['latitude'].iloc[i+1]

    fig.add_annotation(
    xref="x",
    yref="y",
    x=lon2,
    y=lat2,
    text="Pino",
    # If axref is exactly the same as xref, then the text's position is
    # absolute and specified in the same coordinates as xref.
    axref="x domain",
    # The same is the case for yref and ayref, but here the coordinates are data
    # coordinates
    ayref="y",
    ax=lon1,
    ay=lat2,
    arrowhead=2,
    )

    
 


fig.update_geos(fitbounds="locations")
fig.update_layout(
    geo=dict(
        showframe=True,  # Show frame around the map
        showcoastlines=True,  # Show coastlines
        showland=True,  # Show land
        landcolor="LightGreen",
        countrycolor="DarkBlue",
        coastlinecolor="RebeccaPurple",
        projection=dict(type="equirectangular"),
        #lonaxis=dict(range=[min_lon - lon_expand, max_lon + lon_expand]), 
        #lataxis=dict(range=[min_lat - lat_expand, max_lat + lat_expand])
    )
)

# Customize your map background here

fig.show()

In [ ]:
import geopandas as gpd
import plotly.graph_objs as go
import math

# Assuming 'gdf' is your GeoDataFrame
best_matches_gdf['longitude'] = best_matches_gdf['geometry'].x
best_matches_gdf['latitude'] = best_matches_gdf['geometry'].y

# Initialize a figure
fig = go.Figure()

# Add scatter plot for points
fig.add_trace(go.Scattergeo(
    lon=best_matches_gdf['longitude'],
    lat=best_matches_gdf['latitude'],
    mode='markers',
    hoverinfo='text'
))

# Function to calculate rotation angle
def calculate_angle(lon1, lat1, lon2, lat2):
    angle = math.atan2(lat2 - lat1, lon2 - lon1)
    return math.degrees(angle)

# Add lines and triangle annotations
for i in range(len(best_matches_gdf) - 1):
    lon1, lat1 = best_matches_gdf['longitude'].iloc[i], best_matches_gdf['latitude'].iloc[i]
    lon2, lat2 = best_matches_gdf['longitude'].iloc[i+1], best_matches_gdf['latitude'].iloc[i+1]
    
    # Draw line
    fig.add_trace(
        go.Scattergeo(
            lon=[lon1, lon2],
            lat=[lat1, lat2],
            mode='lines',
            line=dict(width=2, color='blue'),
        )
    )
    
    # Calculate angle for the arrow
    angle = calculate_angle(lon1, lat1, lon2, lat2)
    
    # Add rotated triangle (arrow)
    fig.add_annotation(
        x=lon2, y=lat2,
        xref="x", yref="y",
        text="▲", # Triangle symbol as arrow
        showarrow=False,
        font=dict(size=12),
        xanchor="center",
        yanchor="middle",
        align="center",
        textangle=angle
    )

# Update layout
fig.update_layout(
    title='Journey Plot',
    showlegend=False,
    geo=dict(
        scope='world',
        projection_type='equirectangular',
    ),
)

fig.show()


In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import geopandas as gpd
import math

# Assuming 'best_matches_gdf' is your GeoDataFrame
best_matches_gdf['longitude'] = best_matches_gdf.geometry.x
best_matches_gdf['latitude'] = best_matches_gdf.geometry.y

fig.update_layout(xaxis=dict(domain=[0, 0.5]), yaxis=dict(domain=[0.25, 0.75]))

# Create initial scatter plot
fig = px.scatter_geo(best_matches_gdf, lat='latitude', lon='longitude', hover_name='description', text='place_in_json')
fig.update_traces(textposition='bottom center')

# Add lines and annotations
for i in range(len(best_matches_gdf) - 1):
    lon1, lat1 = best_matches_gdf['longitude'].iloc[i], best_matches_gdf['latitude'].iloc[i]
    lon2, lat2 = best_matches_gdf['longitude'].iloc[i+1], best_matches_gdf['latitude'].iloc[i+1]

    # Adding lines
    fig.add_trace(go.Scattergeo(
        lon=[lon1, lon2],
        lat=[lat1, lat2],
        mode='lines',
        line=dict(width=2, color='blue')
    ))

    # Adding annotations
    fig.add_annotation(
        x=lon2,
        y=lat2,
        ax=lon1,
        ay=lat1,
        xref="paper",
        yref="paper",
        axref="x",
        ayref="y",
        showarrow=True,
        arrowhead=2,
    )

# Update layout
fig.update_layout(
    geo=dict(
        showframe=True,
        showcoastlines=True,
        showland=True,
        landcolor="LightGreen",
        countrycolor="DarkBlue",
        coastlinecolor="RebeccaPurple",
        projection=dict(type="equirectangular")
    )
)
fig.update_geos(fitbounds="locations")

# Show the figure
fig.show()


In [ ]:
import plotly.express as px
import plotly.graph_objs as go
import geopandas as gpd
import math

# Assuming 'best_matches_gdf' is your GeoDataFrame
final_gdf['longitude'] = final_gdf.geometry.x
final_gdf['latitude'] = final_gdf.geometry.y


# Create initial scatter plot
fig = px.scatter_geo(final_gdf, lat='latitude', lon='longitude', hover_name='description', text='Location JSON Name')
fig.update_traces(textposition='bottom center')

mid_lons = []
mid_lats = []

def calculate_angle(lon1, lat1, lon2, lat2):
    angle = math.atan2(lat2 - lat1, lon2 - lon1)
    return math.degrees(angle)

def calculate_bearing(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Difference in coordinates
    d_lon = lon2 - lon1

    # Calculate bearing
    x = math.sin(d_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))

    # Obtain bearing in degrees
    bearing = math.degrees(math.atan2(x, y))

    # Normalize the bearing
    bearing = (bearing + 360) % 360

    return bearing


# Add lines and annotations
for i in range(len(final_gdf) - 1):
    lon1, lat1 = final_gdf['longitude'].iloc[i], final_gdf['latitude'].iloc[i]
    lon2, lat2 = final_gdf['longitude'].iloc[i+1], final_gdf['latitude'].iloc[i+1]

    angle = 0
    mid_lon = (lon1 + lon2) / 2
    mid_lat = (lat1 + lat2) / 2
    angle = calculate_angle(lon1, lat1, lon2, lat2)
    bearing = calculate_bearing(lat1, lon1, lat2, lon2)

    # Adding lines
    fig.add_trace(go.Scattergeo(
        lon=[lon1, lon2],
        lat=[lat1, lat2],
        mode='lines+text',
        name=i+1,
        line=dict(width=2, color='blue'),
        hovertext=final_gdf['description'].iloc[i],
        opacity=1,
        showlegend=False
        
        )
    )

    fig.add_trace(go.Scattergeo(
        lon=[mid_lon],
        lat=[mid_lat],
        mode='text+markers',
        name=i+1,
        hovertext=final_gdf['description'].iloc[i],
        opacity=1,
        showlegend=False,


        text="",  # Assuming 'label' is a column with labels for lines
        textposition='middle center',
        textfont=dict(
        size=14,
        color='red',  # Change to your preferred color
        family='Arial'  # Specify font family and style
    ),
        
        marker=dict(
        color='black',  # Black background
        opacity=1,
        size=14,
        symbol='arrow-wide',
        angle=bearing,
    ),
        )
    )

    #mid_lons.append(mid_lon)
    #mid_lats.append(mid_lat)

    #px.scatter(mid_lats, mid_lons, text='pino')
    








fig.update_geos(fitbounds="locations")

# Show the figure
fig.show()
